---
title: "Bayesian Agents"
description: "Developing Bayesian agents for game theory"
date: 2024-06-01
categories: 
  - game theory
  - bayesian agents
  - Agent Based Modeling
  - ABM
  - MESA
  - bayesian game theory
draft: true
---


I want to create a Bayesian and updating scheme for Lewis signaling games that supports fast learning of signaling systems.

One direction is to use hierarchical model. 
First I wanted to draw the initial wights from a prior 
is there a prior one can use for hierarchical learning in the Lewis signaling game.

the name of a prior for an distribution that is like an identity matrix?


Some thought on modeling games with agents.

1. Ideally one should be able to plug in a minimal amount of information about the agents and then be able to 
simulate the game and identify the optimal strategies for the agents.
2. One should be able to simulate the game with different solution concepts and see how the agents behave - like making mistakes or introducing private information.

For example for two player games we can provide a payoff matrix.
then we can simulate the players playing the game in turn or at the same time
once and repeatedly (with memory) and see how the agents behave.

This could cover a wide range of games from the prisoner's dilemma, stag hunt, battle of the sexes, Lewis signaling game with two signals.
A simple bandit algorithm could be used to simulate the agents playing the game and identify the optimal strategies for the agents.


## prisoners dilemma

the payoff matrix for the prisoners dilemma is:

In [ ]:
import numpy as np

prisoners_dillema_payoff_matrix = np.array([[(-1, -1), (-3, 0)], [(0, -3), (-2, -2)]])
stag_hunt_payoff_matrix = np.array([[(-1, -1), (-3, 0)], [(0, -3), (-2, -2)]])
lewis_signaling_game_payoff_matrix = np.array([[(1, 1), (0, 0)], [(0, 0), (1, 1)]])
battle_of_the_sexes_payoff_matrix = np.array([[(2, 1), (0, 0)], [(0, 0), (1, 2)]])
dove_hawk_payoff_matrix = np.array([[(3, 3), (0, 4)], [(4, 0), (1, 1)]])
suppot_oppose_evade_payoff_matrix = np.array([[(6, 4), (2, 8),(8,2)],
                                              [(8, 2), (25, 7.5),(7.5,2.5)], 
                                              [(3.5, 6.5), (3, 7),(4,6)]])
chicken_payoff_matrix = np.array([[(0, 0), (-1, 1)], [(1, -1), (-10, -10)]])
a=100
b=10
robber_guards_payoff_matrix = np.array([[(0, 0), (a, -1*a)], [(b, -1*b), (0, 0)]])  # mixed stategy

For games with incomplete information we can provide a prior distribution over the possible payoffs and then update the distribution based on the agents actions.


ion exploring the space of possible games and strategies
one should be able to identify the optimal strategies for the agents.
2. 


Some thoughts on developing the Bayesian agents:

Pareto improvement
:   In welfare economics, a Pareto improvement formalizes the idea of an outcome being "better in every possible way". A change is called a Pareto improvement if it leaves everyone in a society better-off (or at least as well-off as they were before). 

Pareto efficient or Pareto optimality
:   A situation is called Pareto efficient or Pareto optimal if all possible Pareto improvements have already been made; in other words, there are no longer any ways left to make one person better-off, unless we are willing to make some other person worse-off

Multi-objective optimization or Pareto optimization 
: is an area of multiple-criteria decision making that is concerned with mathematical optimization problems involving more than one objective function to be optimized simultaneously.

Admissible decision rule
:   In statistical decision theory, an admissible decision rule is a rule for making a decision such that there is no other rule that is always "better" than it, in the precise sense of "better" defined below. This concept is analogous to Pareto efficiency.

e.g. The James–Stein estimator is a nonlinear estimator of the mean of Gaussian random vectors and can be shown to dominate the ordinary least squares technique with respect to a mean-squared-error loss function. Therefore in this context the James–Stein estimator is admissible, while the ordinary least squares estimator is inadmissible.

## Hierarchy of solution concepts

1. What is the hierarchy of solution concepts - in the sense that one solution concept can provide better solutions for a broader class of games than another?

One of the tricky aspects is that games can seem very different at first yet 
when we work out the optimal strategies, it turns out that the crucial aspects 
of the games are the same.

Solution concepts typically apply to a given class of games and these classes 
can be used to provide a multidimensional hierarchy of solution concepts.

Strict dominance, weak dominance, iterated dominance, Nash equilibrium, correlated equilibrium, sub-game perfect equilibrium, Bayesian Nash equilibrium, trembling hand perfect equilibrium, sequential equilibrium, perfect Bayesian equilibrium, 

pareto optimality, ESS, backward induction, minimax, maxmin, risk dominance, quantal response equilibrium, level-k reasoning, cognitive hierarchy, iterated elimination of dominated strategies, rationalizability, sequential equilibrium, trembling hand perfect equilibrium, proper equilibrium, sequential equilibrium, perfect Bayesian equilibrium,
core, Shapley value, nucleolus, kernel, bargaining set, von Neumann-Morgenstern solution, Nash bargaining solution, Kalai-Smorodinsky solution, egalitarian solution, competitive equilibrium, Walrasian equilibrium, Arrow-Debreu equilibrium, Radner, 

for non-coopertaive game: Mertens stable equilibrium > forward induction, backward induction 

1. Given a set of agent, with a schedule, action and payoff - can we define a 'formal models' for game in extensive and normal form.
2. For the formal game can we identifying all the different equlibria for a game is specified?
3. Implementing different solution concepts for game theoretic agents.
4. For games with incomplete information, can we implement a bayesian updating scheme for agents.
5. Can we implement a learning scheme for agents in a game.


In [ ]:
import warnings
from warnings import simplefilter
warnings.filterwarnings('ignore', message='The AgentSet is experimental*')

# Import necessary modules
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import numpy as np

class BayesianUpdater:
    def __init__(self, prior=None):
        if prior is None:
            # Default prior: uniform distribution over actions 'A' and 'B'
            prior = {'A': 0.5, 'B': 0.5}
        self.prior = prior
        self.belief = prior.copy()

    def update_belief(self, observation, likelihoods):
        # Update belief using Bayesian updating for each action
        for action in self.belief:
            self.belief[action] *= likelihoods[action]
        
        # Normalize to get new belief
        total = sum(self.belief.values())
        for action in self.belief:
            self.belief[action] /= total

    def make_decision(self):
        # Example decision rule: choose action with highest belief
        return max(self.belief, key=self.belief.get)

class BayesianAgent(Agent):
    def __init__(self, unique_id, model, prior=None):
        super().__init__(unique_id, model)
        self.bayesian_updater = BayesianUpdater(prior)
        self.observed_actions = []
        self.action = None

    def step(self):
        # Make a decision based on current belief
        self.action = self.bayesian_updater.make_decision()
        
        # Update belief based on the observed outcome
        observation = self.model.observe(self)
        likelihoods = {action: self.model.likelihood(observation, action) for action in self.bayesian_updater.prior}
        self.bayesian_updater.update_belief(observation, likelihoods)
        
        # Observe actions of all other agents
        self.observe_other_agents()
        
        # Print detailed output
        print(f"Agent {self.unique_id} action: {self.action}")
        print(f"Agent {self.unique_id} belief: {self.bayesian_updater.belief}")

    def observe_other_agents(self):
        # Observe actions of all other agents in the model
        self.observed_actions = [agent.action for agent in self.model.schedule.agents if agent != self]
        print(f"Agent {self.unique_id} observed actions: {self.observed_actions}")

    def update_belief_about_others(self):
        # Update belief about the world based on observed actions
        for action in self.observed_actions:
            likelihoods = {'A': self.model.likelihood(True, 'A'), 'B': self.model.likelihood(True, 'B')}
            self.bayesian_updater.update_belief(True, likelihoods)

class BayesianModel(Model):
    def __init__(self, N):
        super().__init__()
        self.num_agents = N
        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(10, 10, True)
        
        # Define priors for three types of agents
        prior_type_1 = {'A': 0.8, 'B': 0.2}
        prior_type_2 = {'A': 0.5, 'B': 0.5}
        prior_type_3 = {'A': 0.2, 'B': 0.8}

        # Create agents with different priors
        for i in range(self.num_agents):
            if i % 3 == 0:
                prior = prior_type_1
            elif i % 3 == 1:
                prior = prior_type_2
            else:
                prior = prior_type_3

            agent = BayesianAgent(i, self, prior)
            self.schedule.add(agent)
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(agent, (x, y))
        
        self.datacollector = DataCollector(
            agent_reporters={"Belief": lambda a: a.bayesian_updater.belief}
        )

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()
        for agent in self.schedule.agents:
            agent.update_belief_about_others()

    def observe(self, agent):
        # Simulate an observation based on the agent's action
        if agent.action == 'A':
            return self.random.random() < 0.7  # 70% chance of success
        else:
            return self.random.random() < 0.3  # 30% chance of success

    def likelihood(self, observation, action):
        # Return likelihood of observation given action
        if action == 'A':
            return 0.7 if observation else 0.3
        else:
            return 0.3 if observation else 0.7

# Run the model
if __name__ == "__main__":
    model = BayesianModel(10)
    for i in range(10):  # Reduced the number of steps for brevity
        print(f"\n--- Step {i + 1} ---")
        model.step()
    
    # Extract and print data
    data = model.datacollector.get_agent_vars_dataframe()
    print(data.tail())